# Bottle Processing
Author: Andrew Reed

### Motivation:
Independent verification of the suite of physical and chemical observations provided by OOI are critical for the observations to be of use for scientifically valid investigations. Consequently, CTD casts and Niskin water samples are made during deployment and recovery of OOI platforms, vehicles, and instrumentation. The water samples are subsequently analyzed by independent labs for  comparison with the OOI telemetered and recovered data.

However, currently the water sample data routinely collected and analyzed as part of the OOI program are not available in a standardized format which maps the different chemical analyses to the physical measurements taken at bottle closure. Our aim is to make these physical and chemical analyses of collected water samples available to the end-user in a standardized format for easy comprehension and use, while maintaining the source data files. 

### Approach:
Generating a summary of the water sample analyses involves preprocessing and concatenating multiple data sources, and accurately matching samples with each other. To do this, I first preprocess the ctd casts to generate bottle (.btl) files using the SeaBird vendor software following the SOP available on Alfresco. 

Next, the bottle files are parsed using python code and the data renamed following SeaBird's naming guide. This creates a series of individual cast summary (.sum) files. These files are then loaded into pandas dataframes, appended to each other, and exported as a csv file containing all of the bottle data in a single data file.

### Data Sources/Software:

* **sbe_name_map**: This is a spreadsheet which maps the short names generated by the SeaBird SBE DataProcessing Software to the associated full names. The name mapping originates from SeaBird's SBE DataProcessing support documentation.

* **Alfresco**: The Alfresco CMS for OOI at alfresco.oceanobservatories.org is the source of the ctd hex, xmlcon, and psa files necessary for generating the bottle files needed to create the sample summary sheet.

* **SBEDataProcessing-Win32**: SeaBird vendor software for processing the raw ctd files and generating the .btl files.


In [1]:
# Import packages used in this notebook
import os, sys, re
import pandas as pd
import numpy as np

In [2]:
# Load the name mapping for the column names
# Specifiy the local directory
sbe_name_map = pd.read_excel('/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')
# sbe_name_map = pd.read_excel('C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')

In [401]:
basepath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/'
array = 'Pioneer/'
cruise = 'Pioneer-11/'
leg = 'AR31-C/'

In [ ]:
def clean_sal_files(dirpath):
    
    # Run check if files are held in excel format or csvs
    csv_flag = any(files.endswith('.SAL') for files in os.listdir(dirpath))
    if csv_flag:
        for files in os.listdir(dirpath):
            sample = []
            salinity = []
            with open(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg+files) as file:
                data = file.readlines()
                for ind1,line in enumerate(data):
                    if ind1 == 0:
                        strs = data[0].replace('"','').split(',')
                        cruisename = strs[0]
                        station = strs[1]
                        cast = strs[2]
                        case = strs[8]
                    elif int(line.split()[0]) == 0:
                        pass
                    else:
                        strs = line.split()
                        sample.append(strs[0])
                        salinity.append(strs[2]) 
                # Generate a pandas dataframe to populate data
                data_dict = {'Cruise ID':cruisename,'Station #':station,'Cast #':cast,'Case':case,'Sample ID':sample,'Salinity [psu]':salinity}
                df = pd.DataFrame.from_dict(data_dict)
                df.to_csv(file.name.replace('.','')+'.csv')
    
    else:
        # If the files are already in excel spreadsheets, they've been cleaned into a
        # logical tabular format
        pass
    
def concat_sal_files(dirpath):
        

In [409]:
any(files.endswith('.SAL') for files in os.listdir(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg))

False

In [402]:
os.listdir(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg)

['001OXY.xlsx',
 '001SAL.xlsx',
 '002OXY.xlsx',
 '002SAL.xlsx',
 '004OXY.xlsx',
 '004SAL.xlsx',
 '011OXY.xlsx',
 '011SAL.xlsx',
 '012OXY.xlsx',
 '012SAL.xlsx',
 '020OXY.xlsx',
 '020SAL.xlsx']

In [394]:
for files in os.listdir(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg):
    if '.SAL' in files:
        sample = []
        salinity = []
        with open(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg+files) as file:
            data = file.readlines()
            for ind1,line in enumerate(data):
                if ind1 == 0:
                    strs = data[0].replace('"','').split(',')
                    cruisename = strs[0]
                    station = strs[1]
                    cast = strs[2]
                    case = strs[8]
                elif int(line.split()[0]) == 0:
                    pass
                else:
                    strs = line.split()
                    sample.append(strs[0])
                    salinity.append(strs[2]) 
            # Generate a pandas dataframe to populate data
            data_dict = {'Cruise ID':cruisename,'Station #':station,'Cast #':cast,'Case':case,'Sample ID':sample,'Salinity [psu]':salinity}
            df = pd.DataFrame.from_dict(data_dict)
            df.to_csv(file.name.replace('.','')+'.csv')
        

In [376]:
df

,Cruise ID,Station #,Cast #,Case,Sample ID,Salinity [psu]
0,SHORE,002,1,A,1,35.7023
1,SHORE,002,1,A,2,35.7081
2,SHORE,002,1,A,3,35.0547
3,SHORE,002,1,A,4,35.0565
4,SHORE,002,1,A,5,35.1368
5,SHORE,002,1,A,6,35.1370
6,SHORE,002,1,A,7,34.9107
7,SHORE,002,1,A,8,34.8330
0,SHORE,003,1,A,1,35.6676
1,SHORE,003,1,A,2,35.6688


In [395]:
df = pd.DataFrame()
for file in os.listdir(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg):
    if 'SAL.csv' in file:
        df = df.append(pd.read_csv(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg+file))
df.dropna(inplace=True)
df['Station #'] = df['Station #'].apply(lambda x: str( int(x)).zfill(3))
df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
df.drop(columns='Unnamed: 0',inplace=True)
df.head()

KeyError: 'Station #'

In [403]:
df = pd.DataFrame()
for file in os.listdir(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg):
    if 'SAL' in file:
        df = df.append(pd.read_excel(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg+file))
df.rename({'Cruise':'Cruise ID','Station':'Station #','Sample':'Sample ID','Salinity':'Salinity [psu]'},
          axis='columns',inplace=True)
df.dropna(inplace=True)
df['Station #'] = df['Station #'].apply(lambda x: str( int(x)).zfill(3))
df['Niskin #'] = df['Niskin #'].apply(lambda x: str( int(x)))
df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
df

,Cruise ID,Station #,Niskin #,Case ID,Sample ID,Salinity [psu],Unit
1,AR31-C,001,1,A,1,35.5091,psu
2,AR31-C,001,2,A,2,35.5127,psu
3,AR31-C,001,3,A,3,34.6183,psu
4,AR31-C,001,4,A,4,34.6187,psu
5,AR31-C,001,5,A,5,32.8272,psu
6,AR31-C,001,6,A,6,32.8276,psu
7,AR31-C,001,7,A,7,32.7257,psu
8,AR31-C,001,8,A,8,32.7243,psu
1,AR31-C,002,1,T,1,35.5498,psu
2,AR31-C,002,2,T,2,35.5502,psu


In [404]:
df.to_csv(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg+'SAL_Summary.csv')

In [405]:
o2 = pd.DataFrame()
for file in os.listdir(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg):
    if 'OXY' in file:
        o2 = o2.append(pd.read_excel(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg+file))
o2.head()

,Cruise,Station,Niskin #,Case,Sample#,Oxy,Unit
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AR31-C,1.0,1.0,F,1.0,4.004,mL/L
2,AR31-C,1.0,2.0,F,2.0,3.932,mL/L
3,AR31-C,1.0,3.0,F,3.0,4.528,mL/L
4,AR31-C,1.0,4.0,F,4.0,4.516,mL/L


In [406]:
o2.rename({'Cruise':'Cruise ID','Station':'Station #','Sample#':'Sample ID','Oxy':'Oxygen [mL/L]','Unit':'Units'},
          axis='columns',inplace=True)
o2.dropna(inplace=True)
#o2.drop(0,inplace=True)
o2['Station #'] = o2['Station #'].apply(lambda x: str( int(x)).zfill(3))
o2['Niskin #'] = o2['Niskin #'].apply(lambda x: str( int(x)))
o2['Sample ID'] = o2['Sample ID'].apply(lambda x: str( int(x)))
o2['Cruise ID'] = o2['Cruise ID'].apply(lambda x: x.replace('O','0'))
o2.head()

,Cruise ID,Station #,Niskin #,Case,Sample ID,Oxygen [mL/L],Units
1,AR31-C,001,1,F,1,4.004,mL/L
2,AR31-C,001,2,F,2,3.932,mL/L
3,AR31-C,001,3,F,3,4.528,mL/L
4,AR31-C,001,4,F,4,4.516,mL/L
5,AR31-C,001,5,F,5,5.952,mL/L


In [407]:
o2.to_csv(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg+'OXY_Summary.csv')

In [ ]:
# Specify the local directory where the bottle (.btl) files are stored for a particular cruise
# dirpath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/ctd/'
btlpath = basepath+array+cruise+'KN-214_CTD/'
summary_sheet_path = basepath+array+cruise+'/Water Sampling/Pioneer-1_KN-214_Sampling_Log.xlsx'
salts_and_o2_path = basepath+array+cruise+'/Water Sampling/Irminger_Sea-02_AT-30-01_Oxygen_and_Salnity_Sample_Data/'
nutrients_path = basepath+array+cruise+'/Water Sampling/Irminger_Sea-02_AT-30-01_Nutrients_Sample_Data_2016-09-01_ver_1-00.xlsx'

In [ ]:
# Parse the data for the start_time
def parse_header(header):
    """
    Parse the header of bottle (.btl) files to get critical information
    for the summary spreadsheet.
    
    Args:
        header - an object containing the header of the bottle file as a list of
            strings, split at the newline.
    Returns:
        hdr - a dictionary object containing the start_time, filename, latitude,
            longitude, and cruise id.
    """
    hdr = {}
    for line in header:
        if 'start_time' in line.lower():
            start_time = pd.to_datetime(re.split('= |\[',line)[1])
            hdr.update({'Start Time':start_time.strftime('%Y-%m-%dT%H:%M:%SZ')})
        elif 'filename' in line.lower():
            hex_name = re.split('=',line)[1].strip()
            hdr.update({'Hex name':hex_name})
        elif 'latitude' in line.lower():
            start_lat = re.split('=',line)[1].strip()
            hdr.update({'Start Latitude':start_lat})
        elif 'longitude' in line.lower():
            start_lon = re.split('=',line)[1].strip()
            hdr.update({'Start Longitude':start_lon})
        elif 'cruise id' in line.lower():
            cruise_id = re.split(':',line)[1].strip()
            hdr.update({'Cruise ID':cruise_id})
        else:
            pass
    
    return hdr
        

In [ ]:
# Now write a function to autopopulate the bottle summary sample sheet
files = [x for x in os.listdir(btlpath) if '.btl' in x]
for filename in files:
    filepath = os.path.abspath(btlpath+filename)
    
    # Load the raw content into memory
    with open(filepath) as file:
        content = file.readlines()
    content = [x.strip() for x in content]
    
    # Now parse the file content
    header = []
    columns = []
    data = []
    for line in content:
        if line.startswith('*') or line.startswith('#'):
            header.append(line)
        else:
            try:
                float(line[0])
                data.append(line)
            except:
                columns.append(line)
    
    # Parse the header
    hdr = parse_header(header)
    
    # Parse the column identifiers
    column_dict = {}
    for line in columns:
        for i,x in enumerate(line.split()):
            try:
                column_dict[i] = column_dict[i] + ' ' + x
            except:
                column_dict.update({i:x})
    
    # Parse the bottle data based on the column header locations
    data_dict = {x:[] for x in column_dict.keys()}

    for line in data:
        if line.endswith('(avg)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            for i,x in enumerate(values):
                data_dict[i].append(x)
        elif line.endswith('(sdev)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            data_dict[1].append(values[0])
        else:
            pass
            
    data_dict[1] = [' '.join(item) for item in zip(data_dict[1][::2],data_dict[1][1::2])]
    
    # With the parsed data and column names, match up the data and column
    # based on the location
    results = {}
    for key,item in column_dict.items():
        values = data_dict[key]
        results.update({item:values})
        
    # Put the results into a dataframe
    df = pd.DataFrame.from_dict(results)
        
    # Now add the parsed info from the header files into the dataframe
    for key,item in hdr.items():
        df[key] = item
        
    # Get the cast number
    cast = filename[filename.index('.')-3:filename.index('.')]
    df['Cast #'] = str(cast).zfill(3)
    
    # Generate a filename for the summary file
    outname = filename.split('.')[0] + '.sum'
    
    # Save the results
    df.to_csv(btlpath+outname)

In [ ]:
# Now, for each "summary" file, load and append to each other
df = pd.DataFrame()
for file in os.listdir(btlpath):
    if '.sum' in file:
        df = df.append(pd.read_csv(btlpath+file))
    else:
        pass

In [ ]:
sbe_name_map['Short Name'].apply(lambda x: str(x).lower());

In [ ]:
# Rename the column title using the sbe_name_mapping 
for colname in list(df.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        df.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [ ]:
df.columns.values

In [ ]:
df.rename(columns={'Bottle Position':'Niskin #'},inplace=True)
df['Niskin #'] = df['Niskin #'].apply(lambda x: str( int(x) ) )
df.drop(columns='Unnamed: 0',inplace=True)
df['Cast #'] = df['Cast #'].apply(lambda x: str(x).zfill(3) )

In [ ]:
#df.to_csv(dirpath+'Irminger-5_Summary.csv')

### Oxygen & Salinity 
Now, we need to add the 

In [ ]:
# Need a way to map the oxygen and salinity samples to the CTD samples
# Load the CTD sample logs

In [ ]:
sample_log = pd.read_excel(summary_sheet_path,sheet_name='Summary',header=0)

In [ ]:
sample_log.head()

In [ ]:
# Rename the column headers
sample_log.rename(columns=lambda x: 'Sample Log: ' + x.strip(), inplace=True)
sample_log['Sample Log: Niskin #'] = sample_log['Sample Log: Niskin #'].apply(lambda x: int(x.replace('*','')) if type(x) == str else x )
sample_log['Sample Log: Niskin #'] = sample_log['Sample Log: Niskin #'].apply(lambda x: x if np.isnan(x) else str( int(x) ) )
sample_log['Sample Log: Cast #'] = sample_log['Sample Log: Station-Cast #'].apply(lambda x: str(x).zfill(3))

In [ ]:
set(list(sample_log['Sample Log: Niskin #']))

In [ ]:
# Load the oxygen data
sal_dir = salts_and_o2_path
sal_df = pd.DataFrame()
oxy_df = pd.DataFrame()

for file in os.listdir(sal_dir):
    if 'SAL' in file:
        sal_df = sal_df.append(pd.read_excel(sal_dir+file))
    elif 'OXY' in file:
        oxy_df = oxy_df.append(pd.read_excel(sal_dir+file))

In [ ]:
oxy_df

In [ ]:
sal_df.rename(columns=lambda x: 'SAL: ' + x.strip(), inplace=True)
sal_df.dropna(subset=['SAL: Cruise'], inplace=True)
sal_df.fillna(-999, inplace=True)
sal_df['SAL: Station'] = sal_df['SAL: Station'].apply(lambda x: str( int(x) ).zfill(3) )
sal_df['SAL: Niskin #'] = sal_df['SAL: Niskin #'].apply(lambda x: str( int(x) ) )
sal_df['SAL: Sample'] = sal_df['SAL: Sample'].apply(lambda x: str( int(x) ) )
sal_df['SAL: Sample'] = sal_df['SAL: Case ID'] + sal_df['SAL: Sample']

In [ ]:
oxy_df.rename(columns=lambda x: 'OXY: ' + x.strip(), inplace=True)
oxy_df.dropna(subset=['OXY: Cruise'], inplace=True)
oxy_df.fillna(-999, inplace=True)
oxy_df['OXY: Station'] = oxy_df['OXY: Station'].apply(lambda x: str( int(x) ).zfill(3) )
oxy_df['OXY: Niskin #'] = oxy_df['OXY: Niskin #'].apply(lambda x: str( int(x) ) )
oxy_df['OXY: Sample#'] = oxy_df['OXY: Sample#'].apply(lambda x: str( int(x) ) )
oxy_df['OXY: Sample#'] = oxy_df['OXY: Case'] + oxy_df['OXY: Sample#']

In [ ]:
# Now I need to merge dataframes together
sample_log = sample_log.merge(sal_df, how='left', left_on=['Sample Log: Cast #','Sample Log: Salts Bottle #'], right_on=['SAL: Station','SAL: Sample'])

In [ ]:
sample_log = sample_log.merge(oxy_df, how='left', left_on=['Sample Log: Cast #','Sample Log: Oxygen Bottle #'],
                              right_on=['OXY: Station','OXY: Sample#'])

In [ ]:
os.listdir('Irminger/Irminger-5/Water Sampling/')

In [ ]:
nutrients = pd.read_excel(nutrients_path,header=0)

In [ ]:
nutrient_bottles = sample_log['Sample Log: Nitrate Bottle 1'].str.split(',').apply(pd.Series, 1).stack()
nutrient_bottles.index = nutrient_bottles.index.droplevel(-1)
nutrient_bottles.name = 'Nitrate Bottle #'

In [ ]:
nutrient_bottles

In [ ]:
# Add the nutrient bottle number back into the sample log, and remove the excess '.'
sample_log = sample_log.join(nutrient_bottles)
sample_log['Nitrate Bottle #'] = sample_log['Nitrate Bottle #'].apply(lambda x: x.replace('.','') if type(x) == str else x)
sample_log['Nitrate Bottle #'] = sample_log['Nitrate Bottle #'].apply(lambda x: x.replace(' ','') if type(x) == str else x)

In [ ]:
# Now I can add the nutrient bottle data to the sample log before loading
sample_log = sample_log.merge(nutrients, how='left', left_on='Nitrate Bottle #', right_on='Sample ID')

In [ ]:
sample_log.columns.values

In [ ]:
sample_columns = ['Sample Log: Cast #','Sample Log: Niskin #','SAL: Salinity','OXY: Oxy','Avg: Nitrate+Nitrite [µmol/L]', 'Avg: Ammonium [µmol/L]',
    'Avg: Phosphate [µmol/L]', 'Avg: Silicate [µmol/L]', 'Avg: Nitrite [µmol/L]', 'Avg: Nitrate [µmol/L]']

In [ ]:
result = df.merge(sample_log[sample_columns], how='left',
                  left_on=['Cast #','Niskin #'], right_on=['Sample Log: Cast #','Sample Log: Niskin #'])

In [ ]:
result

In [ ]:
result.columns.values

In [ ]:
sal_unit = '[' + list(set(sal_df['SAL: Unit']) )[0] + ']'
oxy_unit = '[' + list(set(oxy_df['OXY: Unit']) )[0] + ']'
sal_unit, oxy_unit

In [ ]:
result.drop(['Sample Log: Cast #','Sample Log: Niskin #'], axis=1, inplace=True)

In [ ]:
result.rename(columns=lambda x: x.replace('SAL:','Bottle') + ' ' + sal_unit if 'SAL:' in x else x, inplace=True)
result.rename(columns=lambda x: x.replace('OXY:','Bottle') + ' ' + oxy_unit if 'OXY:' in x else x, inplace=True)
result.rename(columns=lambda x: x.replace('Avg:','Bottle') if 'Avg:' in x else x, inplace=True)

In [ ]:
result

In [ ]:
# Replace all of the nans with -999999 and save to a csv
result.fillna(str(-9999999),inplace=True)

In [ ]:
result

In [ ]:
result.to_csv(salts_and_o2_path+'Irminger-3_Summary.csv')

In [ ]:
os.getcwd()